## InfluxDB ##

In [5]:
from influxdb import InfluxDBClient
import json

In [3]:
client = InfluxDBClient(host='influxdb-influxdb.default.svc.cluster.local', port=8086)

In [4]:
dbs= client.get_list_database()

In [5]:
client.switch_database('prices')

In [6]:
measurements = client.get_list_measurements()

In [8]:
results = client.query('select * from ALC_allcoin ORDER by time DESC LIMIT 1')
results.raw

{'statement_id': 0,
 'series': [{'name': 'ALC_allcoin',
   'columns': ['time', 'price_btc', 'price_usd'],
   'values': [['2018-10-22T19:15:54Z', 2.35e-06, 0.0151903118]]}]}

In [10]:
str = str(measurements)

In [11]:
str = str.replace("\'", "\"")

In [12]:
raw_input = json.loads(str)

In [33]:
output_dict = [x for x in raw_input if x['name'] == '611_USD']

In [34]:
output_dict

[{'name': '611_USD'}]

In [15]:
clear_USD = [x for x in measurements 
               if x['name'].endswith('_USD') == False
              ]

In [16]:
clear_BTC = [x for x in clear_USD 
               if x['name'].endswith('_BTC') == False
              ]

In [17]:
clear_BTC1 = [x for x in clear_BTC 
               if x['name'].endswith('coin') 
              ]

In [19]:
type(output_dict)

list

In [49]:
for x in output_dict:
    client.query(('SELECT * from "%s" ORDER by time DESC LIMIT 1') %  x['name'])
    #results = client.query('SELECT * from ALC_allcoin ORDER by time DESC LIMIT 1')
          
results.raw    

{'statement_id': 0,
 'series': [{'name': '611_USD',
   'columns': ['time', 'marketcap', 'price', 'volume'],
   'values': [['2018-10-22T19:40:26Z', 65427, 0.1423848561, 123]]}]}

In [142]:
out = json.dumps(results.raw)
print(out)

{"statement_id": 0, "series": [{"name": "ALC_allcoin", "columns": ["time", "price_btc", "price_usd"], "values": [["2018-10-22T13:07:55Z", 2.35e-06, 0.0153629382]]}]}


## Working with JSON output ##

In [39]:
for x in output_dict:
    results = client.query(('SELECT * from "%s" ORDER by time DESC LIMIT 1') %  x['name'])
    
    points = list(results.get_points())
    print(points)
    data_file = x['name']+'.json'
    with open(data_file, 'w') as outfile:
        json.dump(points, outfile)
# 611_USD.json
#results.raw
#print(points)

[{'time': '2018-10-22T19:40:26Z', 'marketcap': 65427, 'price': 0.1423848561, 'volume': 123}]
[{'time': '2018-10-22T19:40:26Z', 'marketcap': 65427, 'price': 0.1423848561, 'volume': 123}]


In [37]:
results.raw    

{'statement_id': 0,
 'series': [{'name': '611_USD',
   'columns': ['time', 'marketcap', 'price', 'volume'],
   'values': [['2018-10-22T19:35:33Z', 65116, 0.1417095519, 122]]}]}

In [188]:
# save relusts.raw in a json file
data_file = output_dict['name']
with open('data.json', 'w') as outfile:
    json.dump(results.raw, outfile)

In [38]:
# save columns and values in a json file
with open('data.json', 'w') as outfile:
    json.dump(points, outfile)

In [47]:
def read_data(measurements):
    client = InfluxDBClient(host='influxdb-influxdb.default.svc.cluster.local', port=8086)
    client.switch_database('prices')
    
    for x in measurements:
        results = client.query(('SELECT * from "%s" ORDER by time DESC LIMIT 1') %  x['name'])
        points = list(results.get_points())
        #print(points)
        data_file = x['name']+'.json'
        with open(data_file, 'w') as outfile:
            json.dump(points, outfile)

    return points

In [48]:
read_data(output_dict)

[{'time': '2018-10-22T19:46:29Z',
  'marketcap': 65429,
  'price': 0.1423885136,
  'volume': 123}]

## Kafka ##

In [1]:
from kafka import SimpleProducer, KafkaClient


kafka = KafkaClient('blockchain-kafka-kafka.default.svc.cluster.local:9092')
producer = SimpleProducer(kafka)
#producer.send_messages('test', b'async message')

In [2]:
import yaml
import json
from bson import json_util


In [3]:
import kafka as k
k.__version__

'1.4.3'

In [4]:
with open('611_USD.json') as f:
    data = json.load(f)

In [5]:
print(data)

[{'time': '2018-10-22T19:46:29Z', 'marketcap': 65429, 'price': 0.1423885136, 'volume': 123}]


In [6]:
with open('611_USD.json') as f:
    for line in f:
        d = yaml.safe_load(line)
        jd = json.dumps(d)
       
        producer.send_messages('test', json.dumps(jd, indent=4, default=json_util.default).encode('utf-8'))


FailedPayloadsError: FailedPayloadsError

In [8]:
# send 10 messages to kafka
for i in range(10):
    data = { 'tag ': 'blah',
        'name' : 'sam',
        'index' : i,
        'score': 
            {'row1': 100,
             'row2': 200
        }
    }   
    producer.send_messages('test', json.dumps(data, indent=4, default=json_util.default).encode('utf-8'))